<a href="https://colab.research.google.com/github/baochi0212/Optimization/blob/master/clustering.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
#copy the out.csv to this directory 

In [ ]:
import pandas as pd
import sklearn
import plotly.express as px
from sklearn.cluster import KMeans, SpectralClustering
from sklearn.mixture import GaussianMixture
import numpy as np
import json
from copy import deepcopy
import random
# from tsp import TSP

In [ ]:
data = pd.read_csv('out.csv')

In [ ]:
data = data.drop(['Unnamed: 0'], axis=1)

In [ ]:
data

,time,x,y
0,0.226329,11.452374,104.807993
1,0.157432,11.469027,104.707982
2,0.533260,11.490957,104.833844
3,0.782642,11.511254,104.773356
4,0.127372,11.511943,104.877798
...,...,...,...
531,0.296053,11.621318,104.923603
532,0.182931,11.639438,105.008719
533,0.538650,11.662220,104.801737
534,0.541835,11.662220,104.801737


In [ ]:
#df for use clustering with feature x, y
df = data.drop(['time'], axis=1)

In [ ]:
fig = px.scatter(df, x='x', y='y')
fig.show()

In [ ]:
model = KMeans(n_clusters=3, random_state=3)

In [ ]:
model.fit(df)

KMeans(n_clusters=3, random_state=3)

In [ ]:
df['labels'] = model.labels_

In [ ]:
df

,x,y,labels
0,11.452374,104.807993,2
1,11.469027,104.707982,2
2,11.490957,104.833844,2
3,11.511254,104.773356,2
4,11.511943,104.877798,2
...,...,...,...
531,11.621318,104.923603,1
532,11.639438,105.008719,0
533,11.662220,104.801737,1
534,11.662220,104.801737,1


In [ ]:
#use k means for task:
data['labels'] = df['labels']
data

,time,x,y,labels
0,0.226329,11.452374,104.807993,4
1,0.157432,11.469027,104.707982,2
2,0.533260,11.490957,104.833844,2
3,0.782642,11.511254,104.773356,2
4,0.127372,11.511943,104.877798,2
...,...,...,...,...
531,0.296053,11.621318,104.923603,1
532,0.182931,11.639438,105.008719,2
533,0.538650,11.662220,104.801737,1
534,0.541835,11.662220,104.801737,1


In [ ]:
#depot
df.at[0, 'labels'] = 4

In [ ]:
df

,x,y,labels
0,11.452374,104.807993,4
1,11.469027,104.707982,2
2,11.490957,104.833844,2
3,11.511254,104.773356,2
4,11.511943,104.877798,2
...,...,...,...
531,11.621318,104.923603,1
532,11.639438,105.008719,0
533,11.662220,104.801737,1
534,11.662220,104.801737,1


In [ ]:
fig = px.scatter(df, x='x', y='y', color='labels')
fig.show()

In [ ]:
#Spectral Clustering 
#df for use clustering with feature x, y
df = data.drop(['time'], axis=1)
model = SpectralClustering(n_clusters=3, random_state=3)
model.fit(df)

SpectralClustering(n_clusters=3, random_state=3)

In [ ]:
df['labels'] = model.labels_

In [ ]:
#depot
df.at[0, 'labels'] = 4

In [ ]:
fig = px.scatter(df, x='x', y='y', color='labels')
fig.show()

In [ ]:
#GMM
#df for use clustering with feature x, y
df = data.drop(['time'], axis=1)
model = GaussianMixture(n_components=3, random_state=3)

In [ ]:
gm = model.fit(df)

In [ ]:
df['labels'] = gm.predict(df)
#depot
df.at[0, 'labels'] = 4

In [ ]:
fig = px.scatter(df, x='x', y='y', color='labels')
fig.show()

In [ ]:
def distance_matrix(list):
  matrix = np.random.rand(len(list), len(list))
  for i in range(len(list)):
    for j in range(len(list)):
      x1, y1 = list[i]
      x2, y2 = list[j]
      matrix[i, j] = ((x1 - x2)**2 + (y1 - y2)**2)**0.5
  return matrix
 

dict = {}
for i in range(len(data)):
  label = data.iloc[i].labels
  if label in dict:
    dict[label].append((data.iloc[i].x, data.iloc[i].y))
  else:
    dict[label] = [(data.iloc[i].x, data.iloc[i].y)]

for list in dict:
  dict[list] = distance_matrix(dict[list])
  print(dict[list].shape)

(1, 1)
(23, 23)
(151, 151)
(361, 361)


In [ ]:
#DATA 
data

,time,x,y,labels
0,0.678161,11.452374,104.807993,4
1,0.838654,11.469027,104.707982,2
2,0.536095,11.490957,104.833844,2
3,0.729387,11.511254,104.773356,2
4,0.289419,11.511943,104.877798,2
...,...,...,...,...
531,0.700017,11.621318,104.923603,1
532,0.877089,11.639438,105.008719,2
533,0.739801,11.662220,104.801737,1
534,0.906566,11.662220,104.801737,1


In [ ]:
#get the loads
loads = {}
for i in range(len(data)):
  if data.loc[i].labels in loads:
    loads[data.loc[i].labels].append(data.loc[i].time)
  else:
    loads[data.loc[i].labels] = [data.loc[i].time]
print(len(loads[4]))

1


In [ ]:
result = []
#TSP
class TSP:
    def __init__(self, distance_mat, load, houses):
        self.distance_mat = distance_mat
        self.load = load
        self.houses = houses

    def initial_solution(self):
        return [i for i in range(self.houses + 1)]



    def gen_neighbors(self, sol):
    
        neighbor_list = []
        for k in range(6):
            sol_ = deepcopy(sol)
            i, j = np.random.randint(1, self.houses + 1, size=2)
            if i == j:
                if i < self.houses:
                    j = i + 1
                else:
                    j = i - 3
            sol_[i], sol_[j] = sol_[j], sol_[i]
            neighbor_list.append(sol_)
        return neighbor_list
    def cost(self, sol):
        load = 0 
        sol_ = deepcopy(sol)
        sol_.append(0)
        for i in range(len(sol_) - 1):
            a = sol_[i]
            b = sol_[i + 1]
            load += self.load[b] + self.distance_mat[a][b]

        return load
    #SIMULaTED ANNEALING
    def main(self, T_init, T_end, rate):
        curr = self.initial_solution()
        while T_init > T_end:
            neighbor = random.choice(self.gen_neighbors(curr))
            if self.cost(neighbor) < self.cost(curr):
                curr = neighbor
            if self.cost(neighbor) > self.cost(curr):
                if math.exp(-(self.cost(neighbor) - self.cost(curr))/T_init) > random.uniform(0, 1):
                    curr = neighbor

            T_init = T_init/2
        curr.append(0)
        return curr, self.cost(curr)
#APPLY  TO EACH CLUSTERS
for list in dict:
  if list != 4:
    #distance matrix:
    distance_mat = dict[list]
    #load
    load = loads[list]
    tsp_ = TSP(distance_mat, load, len(load) - 1)
    result.append(tsp_.main(1000, 1, 0.5))
result = sorted(result, key=lambda x: x[1])
print('Z', result[-1][1])

Z 183.38056932387445
